In [131]:
import pandas as pd 
import pyodbc
import sqlalchemy
import datetime as dt 
import numpy as np


#Se definen las conexiones

def get_engine():
    connection_string = "mssql+pyodbc://{0}@{2}:{1}@{2}.database.windows.net:{4}/{3}?driver=ODBC+Driver+17+for+SQL+Server".format(
                        'ko_ice_reader',
                        'XXXXXXX',
                        'koice',
                        'ko_ice',
                        1433)
    res = sqlalchemy.engine.create_engine(connection_string, connect_args={'autocommit': True})
    return res
    
def get_query(query):
    engine = get_engine()
    res = pd.read_sql(query, con=engine)
    return res

In [132]:
query = """ 

select distinct tt.store_id, tt.user_id, tt.date_id, tt.establishment_type, tt.establishment_size, tt.first_position_points, tt.first_position_objetives, tt.first_position_cooler_coca, tt.first_position_complishment, tt.is_reported, tt.ice_points, tt.LocationId, YEAR(srl.VisitStartDateTime) as year, MONTH(srl.VisitStartDateTime) as month, DAY(VisitStartDateTime) as day, sl.Question, sl.Response, srl.* 
from dbo.SceneReportList srl 
join dbo.SurveyList sl on sl.SessionUid = srl.SessionUId
join dbo.a_tt_10_CMP_r_traditional_data_01 tt on tt.SessionUid = srl.SessionUid
where YEAR(srl.VisitStartDateTime) = '2021' and srl.LocationClusterName = 'Traditional' and srl.UserRole = 'Auditor' and sl.Question = 'Tipo Enfriador Detallista' and YEAR(CONVERT(varchar, date_id, 103))='2021' and srl.Scene = 'Enfriador' and srl.SubSceneType in ('Enfriador Coca-Cola', 'Enfriador Detallista')

"""

In [133]:
df = pd.DataFrame(get_query(query))
#df = df.sort_values(by="VisitStartDateTime",ascending=False)
df

,store_id,user_id,date_id,establishment_type,establishment_size,first_position_points,first_position_objetives,first_position_cooler_coca,first_position_complishment,is_reported,...,SurveyCancelCallReason,IsQAEnvironment,SalesOfficeCode,SalesOfficeName,IdVisita,SalesGroupCode,SalesGroupName,TradeGroupCode,TradeGroupName,ParentSceneUid
0,100000195,47713,20210312,NO ENREJADO,GRANDE,5.0,5.0,1.0,1.0,1,...,,None,Texcoco,Texcoco,None,MR0025,Texcoco,ABARROTE,ABARROTE,
1,100000504,45848,20210529,ENREJADO,MEDIANO,5.0,5.0,1.0,1.0,0,...,,None,Texcoco,Texcoco,100000504cdmx24@storecheck.com2021-05-29,MR0025,Texcoco,ABARROTE,ABARROTE,
2,10001249,53032,20210222,NO ENREJADO,GRANDE,5.0,5.0,1.0,1.0,1,...,,None,Cuauhtémoc,Cuauhtémoc,10001249cdcuauhtemoc2@storecheck.com2021-02-22,Jefe CUU,Cef - Cuauhtémoc,ABARROTES,ABARROTES,
3,10001249,53032,20210419,NO ENREJADO,GRANDE,5.0,5.0,0.0,1.0,1,...,,None,Cuauhtémoc,Cuauhtémoc,10001249cdcuauhtemoc2@storecheck.com2021-04-19,Jefe CUU,Cef - Cuauhtémoc,ABARROTES,ABARROTES,
4,100000208,45848,20210228,ENREJADO,GRANDE,5.0,5.0,1.0,1.0,1,...,,None,Texcoco,Texcoco,None,MR0025,Texcoco,ABARROTE,ABARROTE,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175362,9019719,56580,20210618,NO ENREJADO,MEDIANO,5.0,5.0,1.0,1.0,1,...,,None,9000260_NOGALES,Nogales,9019719nogales3@storecheck.com2021-06-18,9000260_901,NOGALES,MISCELANEAS Y ESTANQUILLOS,MISCELANEAS Y ESTANQUILLOS,
175363,90249,45790,20210206,NO ENREJADO,GRANDE,5.0,5.0,1.0,1.0,1,...,,None,Quintana Roo,Quintana Roo,None,9000222-86,Cancún Norte,MISCELÁNEA / ESTANQUILLO,MISCELÁNEA / ESTANQUILLO,
175364,90741,45790,20210113,NO ENREJADO,GRANDE,5.0,5.0,1.0,1.0,1,...,,None,Quintana Roo,Quintana Roo,None,9000222-87,Playa del Carmen,ABARROTE,ABARROTE,
175365,99750,45795,20210104,ENREJADO,MEDIANO,5.0,5.0,1.0,1.0,1,...,,None,Yucatán,Yucatán,None,9000222-30,Oriente,MISCELÁNEA / ESTANQUILLO,MISCELÁNEA / ESTANQUILLO,


In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175367 entries, 0 to 175366
Data columns (total 55 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   store_id                      175367 non-null  object        
 1   user_id                       175367 non-null  int64         
 2   date_id                       175367 non-null  int64         
 3   establishment_type            175367 non-null  object        
 4   establishment_size            175367 non-null  object        
 5   first_position_points         175367 non-null  float64       
 6   first_position_objetives      175367 non-null  float64       
 7   first_position_cooler_coca    175367 non-null  float64       
 8   first_position_complishment   175367 non-null  float64       
 9   is_reported                   175367 non-null  int64         
 10  ice_points                    175367 non-null  float64       
 11  LocationId   

In [135]:
df['date_format']=pd.to_datetime(df[['year','month','day']])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175367 entries, 0 to 175366
Data columns (total 56 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   store_id                      175367 non-null  object        
 1   user_id                       175367 non-null  int64         
 2   date_id                       175367 non-null  int64         
 3   establishment_type            175367 non-null  object        
 4   establishment_size            175367 non-null  object        
 5   first_position_points         175367 non-null  float64       
 6   first_position_objetives      175367 non-null  float64       
 7   first_position_cooler_coca    175367 non-null  float64       
 8   first_position_complishment   175367 non-null  float64       
 9   is_reported                   175367 non-null  int64         
 10  ice_points                    175367 non-null  float64       
 11  LocationId   

In [136]:
df.duplicated(subset='SessionUid', keep='first').sum()

125335

In [137]:
df=df.drop_duplicates(subset=['SessionUid'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50032 entries, 0 to 175361
Data columns (total 56 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   store_id                      50032 non-null  object        
 1   user_id                       50032 non-null  int64         
 2   date_id                       50032 non-null  int64         
 3   establishment_type            50032 non-null  object        
 4   establishment_size            50032 non-null  object        
 5   first_position_points         50032 non-null  float64       
 6   first_position_objetives      50032 non-null  float64       
 7   first_position_cooler_coca    50032 non-null  float64       
 8   first_position_complishment   50032 non-null  float64       
 9   is_reported                   50032 non-null  int64         
 10  ice_points                    50032 non-null  float64       
 11  LocationId                 

In [25]:
#df_cc=df.loc[df['SubSceneType'] == 'Enfriador Detallista']
#df_cc.info()

In [138]:
df['first_position_cooler_coca'] = df['first_position_cooler_coca'].astype(int)
df['first_position_complishment'] = df['first_position_complishment'].astype(int)
df.info()

<ipython-input-138-acda2fdef5bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_position_cooler_coca'] = df['first_position_cooler_coca'].astype(int)
<ipython-input-138-acda2fdef5bb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_position_complishment'] = df['first_position_complishment'].astype(int)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50032 entries, 0 to 175361
Data columns (total 56 columns):
 #   Column                        Non-Null Count  Dtype         
---  -----

In [139]:
df=df.drop(columns=['IsQAEnvironment'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50032 entries, 0 to 175361
Data columns (total 55 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   store_id                      50032 non-null  object        
 1   user_id                       50032 non-null  int64         
 2   date_id                       50032 non-null  int64         
 3   establishment_type            50032 non-null  object        
 4   establishment_size            50032 non-null  object        
 5   first_position_points         50032 non-null  float64       
 6   first_position_objetives      50032 non-null  float64       
 7   first_position_cooler_coca    50032 non-null  int32         
 8   first_position_complishment   50032 non-null  int32         
 9   is_reported                   50032 non-null  int64         
 10  ice_points                    50032 non-null  float64       
 11  LocationId                 

In [144]:
table = pd.pivot_table(df, values=['first_position_complishment'], index=['UserName', 'OutletCode', 'user_id', 'date_id', 'is_reported', 'date_format', 'month', 'day', 'Question', 'Response', 'SessionUid'], columns=['SubSceneType'], aggfunc={'first_position_complishment':len},
                     fill_value=0)
table=pd.DataFrame(table)
table

first_position_complishment  \
SubSceneType                                                                                                                                                                     Enfriador Coca-Cola   
UserName                OutletCode user_id date_id  is_reported date_format month day Question                  Response            SessionUid                                                         
acambaro@storecheck.com 1601006979 45806   20210119 1           2021-01-19  1     19  Tipo Enfriador Detallista Vitrina            3f5f3a80-dff7-4c99-adee-c63d40b0bc48                           1   
                        1601007959 45806   20210119 1           2021-01-19  1     19  Tipo Enfriador Detallista Vitrina            26580785-1a49-4a1f-b976-ce46c4975293                           1   
                        1601011944 45806   20210121 1           2021-01-21  1     21  Tipo Enfriador Detallista Vitrina            6e95d49d-1298-44c1-a677-943a5bd231b7                           0   
                                           20210223 1           2021-02-23  2     23  Tipo Enfriador Detallista Vitrina            2cd5f926-0e77-42ce-847f-475e74315e68                           0   
                                           20210325 1           2021-03-25  3     25  Tipo Enfriador Detallista Vitrina            003ac733-f475-4b08-bd66-741e3e33bc1a                           0   
...                                                                                                                                                                                              ...   
zamora@storecheck.com   620150075  45818   20210604 1           2021-06-04  6     4   Tipo Enfriador Detallista Enfriador Vertical 5fe143f1-1613-43ee-8514-d7e66a48a0c3                           1   
                        620161330  45818   20210219 1           2021-02-19  2     19  Tipo Enfriador Detallista Enfriador Vertical 225d786c-a7de-4a13-a116-9e8f62fe7925                           1   
                                           20210319 1           2021-03-19  3     19  Tipo Enfriador Detallista Enfriador Vertical 35ea8575-e115-42eb-82a3-01edd0308a79                           1   
                                           20210513 1           2021-05-13  5     13  Tipo Enfriador Detallista Enfriador Vertical a29b1b45-d4d3-4add-b890-daaaaabb42f4                           1   
                                           20210609 1           2021-06-09  6     9   Tipo Enfriador Detallista Enfriador Vertical ad151b20-d39b-4fb8-a217-421e97536e9a                           1   

                                                                                                                                                                                               
SubSceneType                                                                                                                                                             Enfriador Detallista  
UserName                OutletCode user_id date_id  is_reported date_format month day Question                  Response            SessionUid                                                 
acambaro@storecheck.com 1601006979 45806   20210119 1           2021-01-19  1     19  Tipo Enfriador Detallista Vitrina            3f5f3a80-dff7-4c99-adee-c63d40b0bc48                    0  
                        1601007959 45806   20210119 1           2021-01-19  1     19  Tipo Enfriador Detallista Vitrina            26580785-1a49-4a1f-b976-ce46c4975293                    0  
                        1601011944 45806   20210121 1           2021-01-21  1     21  Tipo Enfriador Detallista Vitrina            6e95d49d-1298-44c1-a677-943a5bd231b7                    1  
                                           20210223 1           2021-02-23  2     23  Tipo Enfriador Detallista Vitrina            2cd5f926-0e77-42ce-847f-475e74315e68                    1  
                        

In [145]:
table = table.reset_index()
table

UserName  OutletCode user_id   date_id  \
SubSceneType                                                          
0             acambaro@storecheck.com  1601006979   45806  20210119   
1             acambaro@storecheck.com  1601007959   45806  20210119   
2             acambaro@storecheck.com  1601011944   45806  20210121   
3             acambaro@storecheck.com  1601011944   45806  20210223   
4             acambaro@storecheck.com  1601011944   45806  20210325   
...                               ...         ...     ...       ...   
50027           zamora@storecheck.com   620150075   45818  20210604   
50028           zamora@storecheck.com   620161330   45818  20210219   
50029           zamora@storecheck.com   620161330   45818  20210319   
50030           zamora@storecheck.com   620161330   45818  20210513   
50031           zamora@storecheck.com   620161330   45818  20210609   

             is_reported date_format month day                   Question  \
SubSceneType                                                                
0                      1  2021-01-19     1  19  Tipo Enfriador Detallista   
1                      1  2021-01-19     1  19  Tipo Enfriador Detallista   
2                      1  2021-01-21     1  21  Tipo Enfriador Detallista   
3                      1  2021-02-23     2  23  Tipo Enfriador Detallista   
4                      1  2021-03-25     3  25  Tipo Enfriador Detallista   
...                  ...         ...   ...  ..                        ...   
50027                  1  2021-06-04     6   4  Tipo Enfriador Detallista   
50028                  1  2021-02-19     2  19  Tipo Enfriador Detallista   
50029                  1  2021-03-19     3  19  Tipo Enfriador Detallista   
50030                  1  2021-05-13     5  13  Tipo Enfriador Detallista   
50031                  1  2021-06-09     6   9  Tipo Enfriador Detallista   

                         Response                            SessionUid  \
SubSceneType                                                              
0                        Vitrina  3f5f3a80-dff7-4c99-adee-c63d40b0bc48   
1                        Vitrina  26580785-1a49-4a1f-b976-ce46c4975293   
2                        Vitrina  6e95d49d-1298-44c1-a677-943a5bd231b7   
3                        Vitrina  2cd5f926-0e77-42ce-847f-475e74315e68   
4                        Vitrina  003ac733-f475-4b08-bd66-741e3e33bc1a   
...                           ...                                   ...   
50027         Enfriador Vertical  5fe143f1-1613-43ee-8514-d7e66a48a0c3   
50028         Enfriador Vertical  225d786c-a7de-4a13-a116-9e8f62fe7925   
50029         Enfriador Vertical  35ea8575-e115-42eb-82a3-01edd0308a79   
50030         Enfriador Vertical  a29b1b45-d4d3-4add-b890-daaaaabb42f4   
50031         Enfriador Vertical  ad151b20-d39b-4fb8-a217-421e97536e9a   

             first_position_complishment                       
SubSceneType         Enfriador Coca-Cola Enfriador Detallista  
0                                      1                    0  
1                                      1                    0  
2                                      0                    1  
3                                      0                    1  
4                                      0                    1  
...                                  ...                  ...  
50027                                  1                    0  
50028                                  1                    0  
50029                                  1                    0  
50030                                  1                    0  
50031                                  1                    0  

[50032 rows x 13 columns]

In [146]:
table.dtypes


                             SubSceneType        
UserName                                                     object
OutletCode                                                   object
user_id                                                       int64
date_id                                                       int64
is_reported                                                   int64
date_format                                          datetime64[ns]
month                                                         int64
day                                                           int64
Question                                                     object
Response                                                     object
SessionUid                                                   object
first_position_complishment  Enfriador Coca-Cola              int64
                             Enfriador Detallista             int64
dtype: object

In [147]:
df1 = pd.DataFrame(data=table)
df1

UserName  OutletCode user_id   date_id  \
SubSceneType                                                          
0             acambaro@storecheck.com  1601006979   45806  20210119   
1             acambaro@storecheck.com  1601007959   45806  20210119   
2             acambaro@storecheck.com  1601011944   45806  20210121   
3             acambaro@storecheck.com  1601011944   45806  20210223   
4             acambaro@storecheck.com  1601011944   45806  20210325   
...                               ...         ...     ...       ...   
50027           zamora@storecheck.com   620150075   45818  20210604   
50028           zamora@storecheck.com   620161330   45818  20210219   
50029           zamora@storecheck.com   620161330   45818  20210319   
50030           zamora@storecheck.com   620161330   45818  20210513   
50031           zamora@storecheck.com   620161330   45818  20210609   

             is_reported date_format month day                   Question  \
SubSceneType                                                                
0                      1  2021-01-19     1  19  Tipo Enfriador Detallista   
1                      1  2021-01-19     1  19  Tipo Enfriador Detallista   
2                      1  2021-01-21     1  21  Tipo Enfriador Detallista   
3                      1  2021-02-23     2  23  Tipo Enfriador Detallista   
4                      1  2021-03-25     3  25  Tipo Enfriador Detallista   
...                  ...         ...   ...  ..                        ...   
50027                  1  2021-06-04     6   4  Tipo Enfriador Detallista   
50028                  1  2021-02-19     2  19  Tipo Enfriador Detallista   
50029                  1  2021-03-19     3  19  Tipo Enfriador Detallista   
50030                  1  2021-05-13     5  13  Tipo Enfriador Detallista   
50031                  1  2021-06-09     6   9  Tipo Enfriador Detallista   

                         Response                            SessionUid  \
SubSceneType                                                              
0                        Vitrina  3f5f3a80-dff7-4c99-adee-c63d40b0bc48   
1                        Vitrina  26580785-1a49-4a1f-b976-ce46c4975293   
2                        Vitrina  6e95d49d-1298-44c1-a677-943a5bd231b7   
3                        Vitrina  2cd5f926-0e77-42ce-847f-475e74315e68   
4                        Vitrina  003ac733-f475-4b08-bd66-741e3e33bc1a   
...                           ...                                   ...   
50027         Enfriador Vertical  5fe143f1-1613-43ee-8514-d7e66a48a0c3   
50028         Enfriador Vertical  225d786c-a7de-4a13-a116-9e8f62fe7925   
50029         Enfriador Vertical  35ea8575-e115-42eb-82a3-01edd0308a79   
50030         Enfriador Vertical  a29b1b45-d4d3-4add-b890-daaaaabb42f4   
50031         Enfriador Vertical  ad151b20-d39b-4fb8-a217-421e97536e9a   

             first_position_complishment                       
SubSceneType         Enfriador Coca-Cola Enfriador Detallista  
0                                      1                    0  
1                                      1                    0  
2                                      0                    1  
3                                      0                    1  
4                                      0                    1  
...                                  ...                  ...  
50027                                  1                    0  
50028                                  1                    0  
50029                                  1                    0  
50030                                  1                    0  
50031                                  1                    0  

[50032 rows x 13 columns]

In [149]:
array_df=df1.values
array_df

array([['acambaro@storecheck.com', '1601006979', 45806, ...,
        '3f5f3a80-dff7-4c99-adee-c63d40b0bc48', 1, 0],
       ['acambaro@storecheck.com', '1601007959', 45806, ...,
        '26580785-1a49-4a1f-b976-ce46c4975293', 1, 0],
       ['acambaro@storecheck.com', '1601011944', 45806, ...,
        '6e95d49d-1298-44c1-a677-943a5bd231b7', 0, 1],
       ...,
       ['zamora@storecheck.com', '620161330', 45818, ...,
        '35ea8575-e115-42eb-82a3-01edd0308a79', 1, 0],
       ['zamora@storecheck.com', '620161330', 45818, ...,
        'a29b1b45-d4d3-4add-b890-daaaaabb42f4', 1, 0],
       ['zamora@storecheck.com', '620161330', 45818, ...,
        'ad151b20-d39b-4fb8-a217-421e97536e9a', 1, 0]], dtype=object)

In [151]:
df2 = pd.DataFrame(array_df, 
             columns=['UserName', 'OutletCode', 'user_id', 'date_id', 'is_reported', 'date_format', 'month', 'day', 'Question', 'Response', 'SessionUid', 'Enfriador Coca-Cola', 'Enfriador Detallista'])
df2.head()

,UserName,OutletCode,user_id,date_id,is_reported,date_format,month,day,Question,Response,SessionUid,Enfriador Coca-Cola,Enfriador Detallista
0,acambaro@storecheck.com,1601006979,45806,20210119,1,2021-01-19,1,19,Tipo Enfriador Detallista,Vitrina,3f5f3a80-dff7-4c99-adee-c63d40b0bc48,1,0
1,acambaro@storecheck.com,1601007959,45806,20210119,1,2021-01-19,1,19,Tipo Enfriador Detallista,Vitrina,26580785-1a49-4a1f-b976-ce46c4975293,1,0
2,acambaro@storecheck.com,1601011944,45806,20210121,1,2021-01-21,1,21,Tipo Enfriador Detallista,Vitrina,6e95d49d-1298-44c1-a677-943a5bd231b7,0,1
3,acambaro@storecheck.com,1601011944,45806,20210223,1,2021-02-23,2,23,Tipo Enfriador Detallista,Vitrina,2cd5f926-0e77-42ce-847f-475e74315e68,0,1
4,acambaro@storecheck.com,1601011944,45806,20210325,1,2021-03-25,3,25,Tipo Enfriador Detallista,Vitrina,003ac733-f475-4b08-bd66-741e3e33bc1a,0,1


In [154]:
ifi=df2.groupby(['UserName', 'OutletCode', 'user_id', 'date_id', 'is_reported', 'date_format', 'SessionUid', 'Question', 'Response', 'month', 'day'])['Enfriador Coca-Cola', 'Enfriador Detallista'].sum()

ifi

<ipython-input-154-0ebd22b5abac>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ifi=df2.groupby(['UserName', 'OutletCode', 'user_id', 'date_id', 'is_reported', 'date_format', 'SessionUid', 'Question', 'Response', 'month', 'day'])['Enfriador Coca-Cola', 'Enfriador Detallista'].sum()


Enfriador Coca-Cola  \
UserName                OutletCode user_id date_id  is_reported date_format SessionUid                           Question                  Response            month day                        
acambaro@storecheck.com 1601006979 45806   20210119 1           2021-01-19  3f5f3a80-dff7-4c99-adee-c63d40b0bc48 Tipo Enfriador Detallista Vitrina            1     19                     1   
                        1601007959 45806   20210119 1           2021-01-19  26580785-1a49-4a1f-b976-ce46c4975293 Tipo Enfriador Detallista Vitrina            1     19                     1   
                        1601011944 45806   20210121 1           2021-01-21  6e95d49d-1298-44c1-a677-943a5bd231b7 Tipo Enfriador Detallista Vitrina            1     21                     0   
                                           20210223 1           2021-02-23  2cd5f926-0e77-42ce-847f-475e74315e68 Tipo Enfriador Detallista Vitrina            2     23                     0   
                                           20210325 1           2021-03-25  003ac733-f475-4b08-bd66-741e3e33bc1a Tipo Enfriador Detallista Vitrina            3     25                     0   
...                                                                                                                                                                                       ...   
zamora@storecheck.com   620150075  45818   20210604 1           2021-06-04  5fe143f1-1613-43ee-8514-d7e66a48a0c3 Tipo Enfriador Detallista Enfriador Vertical 6     4                      1   
                        620161330  45818   20210219 1           2021-02-19  225d786c-a7de-4a13-a116-9e8f62fe7925 Tipo Enfriador Detallista Enfriador Vertical 2     19                     1   
                                           20210319 1           2021-03-19  35ea8575-e115-42eb-82a3-01edd0308a79 Tipo Enfriador Detallista Enfriador Vertical 3     19                     1   
                                           20210513 1           2021-05-13  a29b1b45-d4d3-4add-b890-daaaaabb42f4 Tipo Enfriador Detallista Enfriador Vertical 5     13                     1   
                                           20210609 1           2021-06-09  ad151b20-d39b-4fb8-a217-421e97536e9a Tipo Enfriador Detallista Enfriador Vertical 6     9                      1   

                                                                                                                                                                          Enfriador Detallista  
UserName                OutletCode user_id date_id  is_reported date_format SessionUid                           Question                  Response            month day                        
acambaro@storecheck.com 1601006979 45806   20210119 1           2021-01-19  3f5f3a80-dff7-4c99-adee-c63d40b0bc48 Tipo Enfriador Detallista Vitrina            1     19                      0  
                        1601007959 45806   20210119 1           2021-01-19  26580785-1a49-4a1f-b976-ce46c4975293 Tipo Enfriador Detallista Vitrina            1     19                      0  
                        1601011944 45806   20210121 1           2021-01-21  6e95d49d-1298-44c1-a677-943a5bd231b7 Tipo Enfriador Detallista Vitrina            1     21                      1  
                                           20210223 1           2021-02-23  2cd5f926-0e77-42ce-847f-475e74315e68 Tipo Enfriador Detallista Vitrina            2     23                      1  
                                           20210325 1           2021-03-25  003ac733-f475-4b08-bd66-741e3e33bc1a Tipo Enfriador Detallista Vitrina            3     25                      1  
...                                                                                                                                                                                        ...  
zamora@storecheck.com   620150075  45818   20210604 1           2021-06-04  5fe143f1-1613-43ee-8514-d7e66a48a0c3 Tip

In [155]:
table2 = ifi.reset_index()
table2

,UserName,OutletCode,user_id,date_id,is_reported,date_format,SessionUid,Question,Response,month,day,Enfriador Coca-Cola,Enfriador Detallista
0,acambaro@storecheck.com,1601006979,45806,20210119,1,2021-01-19,3f5f3a80-dff7-4c99-adee-c63d40b0bc48,Tipo Enfriador Detallista,Vitrina,1,19,1,0
1,acambaro@storecheck.com,1601007959,45806,20210119,1,2021-01-19,26580785-1a49-4a1f-b976-ce46c4975293,Tipo Enfriador Detallista,Vitrina,1,19,1,0
2,acambaro@storecheck.com,1601011944,45806,20210121,1,2021-01-21,6e95d49d-1298-44c1-a677-943a5bd231b7,Tipo Enfriador Detallista,Vitrina,1,21,0,1
3,acambaro@storecheck.com,1601011944,45806,20210223,1,2021-02-23,2cd5f926-0e77-42ce-847f-475e74315e68,Tipo Enfriador Detallista,Vitrina,2,23,0,1
4,acambaro@storecheck.com,1601011944,45806,20210325,1,2021-03-25,003ac733-f475-4b08-bd66-741e3e33bc1a,Tipo Enfriador Detallista,Vitrina,3,25,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50027,zamora@storecheck.com,620150075,45818,20210604,1,2021-06-04,5fe143f1-1613-43ee-8514-d7e66a48a0c3,Tipo Enfriador Detallista,Enfriador Vertical,6,4,1,0
50028,zamora@storecheck.com,620161330,45818,20210219,1,2021-02-19,225d786c-a7de-4a13-a116-9e8f62fe7925,Tipo Enfriador Detallista,Enfriador Vertical,2,19,1,0
50029,zamora@storecheck.com,620161330,45818,20210319,1,2021-03-19,35ea8575-e115-42eb-82a3-01edd0308a79,Tipo Enfriador Detallista,Enfriador Vertical,3,19,1,0
50030,zamora@storecheck.com,620161330,45818,20210513,1,2021-05-13,a29b1b45-d4d3-4add-b890-daaaaabb42f4,Tipo Enfriador Detallista,Enfriador Vertical,5,13,1,0


In [156]:
table2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50032 entries, 0 to 50031
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   UserName              50032 non-null  object        
 1   OutletCode            50032 non-null  object        
 2   user_id               50032 non-null  int64         
 3   date_id               50032 non-null  int64         
 4   is_reported           50032 non-null  int64         
 5   date_format           50032 non-null  datetime64[ns]
 6   SessionUid            50032 non-null  object        
 7   Question              50032 non-null  object        
 8   Response              50032 non-null  object        
 9   month                 50032 non-null  int64         
 10  day                   50032 non-null  int64         
 11  Enfriador Coca-Cola   50032 non-null  int64         
 12  Enfriador Detallista  50032 non-null  int64         
dtypes: datetime64[ns

In [157]:
table2=table2.sort_values(['OutletCode', 'date_format'])

In [158]:
table2["Coca-Cola"] = table2["Enfriador Coca-Cola"].shift(1)
table2["Detallista"] = table2["Enfriador Detallista"].shift(1)
table2.dtypes

UserName                        object
OutletCode                      object
user_id                          int64
date_id                          int64
is_reported                      int64
date_format             datetime64[ns]
SessionUid                      object
Question                        object
Response                        object
month                            int64
day                              int64
Enfriador Coca-Cola              int64
Enfriador Detallista             int64
Coca-Cola                      float64
Detallista                     float64
dtype: object

In [159]:
table2 = table2.fillna(0)
table2

,UserName,OutletCode,user_id,date_id,is_reported,date_format,SessionUid,Question,Response,month,day,Enfriador Coca-Cola,Enfriador Detallista,Coca-Cola,Detallista
12814,cdmx26@storecheck.com,100000195,47713,20210312,1,2021-03-12,78f994cc-4a41-46ab-b0bb-8b11323aac57,Tipo Enfriador Detallista,Vitrina,3,12,1,0,0.0,0.0
11499,cdmx24@storecheck.com,100000208,45848,20210228,1,2021-02-28,1ef2922f-4c61-47ab-a065-e94bc750a97b,Tipo Enfriador Detallista,Vitrina,2,28,1,0,1.0,0.0
12815,cdmx26@storecheck.com,100000208,47713,20210312,1,2021-03-12,1061c10c-0134-4c44-ac84-ba4ae39b248e,Tipo Enfriador Detallista,Vitrina,3,12,0,1,1.0,0.0
5008,cdmx13@storecheck.com,100000283,45836,20210326,1,2021-03-26,dabf8dd8-3a0c-4456-b4ae-0cce7fcadb62,Tipo Enfriador Detallista,Vitrina,3,26,0,1,0.0,1.0
11500,cdmx24@storecheck.com,100000390,45848,20210611,1,2021-06-11,d0b07c3d-e1d8-4674-828f-d5628c9a6c00,Tipo Enfriador Detallista,Vitrina,6,11,0,1,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36256,merida4@storecheck.com,99361,45796,20210122,1,2021-01-22,d9689f04-47bc-4ca1-9748-6d168b1ab902,Tipo Enfriador Detallista,Enfriador Vertical,1,22,1,0,0.0,1.0
36257,merida4@storecheck.com,99361,45796,20210412,1,2021-04-12,fe842bae-8ee8-49fb-b2c1-1c05d5fe7a53,Tipo Enfriador Detallista,Enfriador Vertical,4,12,0,1,1.0,0.0
35654,merida3@storecheck.com,99750,45795,20210104,1,2021-01-04,9537a728-a264-4603-92d8-64e754403dc0,Tipo Enfriador Detallista,Cámara Fría,1,4,0,1,0.0,1.0
36258,merida4@storecheck.com,99750,45796,20210208,1,2021-02-08,01ddcc9b-0294-4658-b0f5-4d004035e31c,Tipo Enfriador Detallista,Enfriador Vertical,2,8,0,1,0.0,1.0


In [161]:
table2 = table2.astype({"Coca-Cola": int, "Detallista": int})
table2

,UserName,OutletCode,user_id,date_id,is_reported,date_format,SessionUid,Question,Response,month,day,Enfriador Coca-Cola,Enfriador Detallista,Coca-Cola,Detallista
12814,cdmx26@storecheck.com,100000195,47713,20210312,1,2021-03-12,78f994cc-4a41-46ab-b0bb-8b11323aac57,Tipo Enfriador Detallista,Vitrina,3,12,1,0,0,0
11499,cdmx24@storecheck.com,100000208,45848,20210228,1,2021-02-28,1ef2922f-4c61-47ab-a065-e94bc750a97b,Tipo Enfriador Detallista,Vitrina,2,28,1,0,1,0
12815,cdmx26@storecheck.com,100000208,47713,20210312,1,2021-03-12,1061c10c-0134-4c44-ac84-ba4ae39b248e,Tipo Enfriador Detallista,Vitrina,3,12,0,1,1,0
5008,cdmx13@storecheck.com,100000283,45836,20210326,1,2021-03-26,dabf8dd8-3a0c-4456-b4ae-0cce7fcadb62,Tipo Enfriador Detallista,Vitrina,3,26,0,1,0,1
11500,cdmx24@storecheck.com,100000390,45848,20210611,1,2021-06-11,d0b07c3d-e1d8-4674-828f-d5628c9a6c00,Tipo Enfriador Detallista,Vitrina,6,11,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36256,merida4@storecheck.com,99361,45796,20210122,1,2021-01-22,d9689f04-47bc-4ca1-9748-6d168b1ab902,Tipo Enfriador Detallista,Enfriador Vertical,1,22,1,0,0,1
36257,merida4@storecheck.com,99361,45796,20210412,1,2021-04-12,fe842bae-8ee8-49fb-b2c1-1c05d5fe7a53,Tipo Enfriador Detallista,Enfriador Vertical,4,12,0,1,1,0
35654,merida3@storecheck.com,99750,45795,20210104,1,2021-01-04,9537a728-a264-4603-92d8-64e754403dc0,Tipo Enfriador Detallista,Cámara Fría,1,4,0,1,0,1
36258,merida4@storecheck.com,99750,45796,20210208,1,2021-02-08,01ddcc9b-0294-4658-b0f5-4d004035e31c,Tipo Enfriador Detallista,Enfriador Vertical,2,8,0,1,0,1


In [162]:
table2.dtypes

UserName                        object
OutletCode                      object
user_id                          int64
date_id                          int64
is_reported                      int64
date_format             datetime64[ns]
SessionUid                      object
Question                        object
Response                        object
month                            int64
day                              int64
Enfriador Coca-Cola              int64
Enfriador Detallista             int64
Coca-Cola                        int32
Detallista                       int32
dtype: object

In [163]:
table2["OutletCodeChanged"] = table2["OutletCode"].shift(1, fill_value=table2["OutletCode"].head(1)) != table2["OutletCode"]
table2.dtypes

UserName                        object
OutletCode                      object
user_id                          int64
date_id                          int64
is_reported                      int64
date_format             datetime64[ns]
SessionUid                      object
Question                        object
Response                        object
month                            int64
day                              int64
Enfriador Coca-Cola              int64
Enfriador Detallista             int64
Coca-Cola                        int32
Detallista                       int32
OutletCodeChanged                 bool
dtype: object

In [164]:
table2["OutletCodeChanged"] = table2["OutletCodeChanged"].astype(object)
table2.dtypes

UserName                        object
OutletCode                      object
user_id                          int64
date_id                          int64
is_reported                      int64
date_format             datetime64[ns]
SessionUid                      object
Question                        object
Response                        object
month                            int64
day                              int64
Enfriador Coca-Cola              int64
Enfriador Detallista             int64
Coca-Cola                        int32
Detallista                       int32
OutletCodeChanged               object
dtype: object

In [165]:
for OutletCodeChanged in table2:
      s = table2['OutletCodeChanged'].unique()
      for i in s:
          if i == 'True':
              Flag_CC = 'change'
              
          else: 
              Flag_CC = table2["Enfriador Coca-Cola"] - table2["Coca-Cola"]

              table2['Flag_CC']=Flag_CC
              
      print(Flag_CC)    

12814    1
11499    0
12815   -1
5008     0
11500    0
        ..
36256    1
36257   -1
35654    0
36258    0
36259    0
Length: 50032, dtype: int64
12814    1
11499    0
12815   -1
5008     0
11500    0
        ..
36256    1
36257   -1
35654    0
36258    0
36259    0
Length: 50032, dtype: int64
12814    1
11499    0
12815   -1
5008     0
11500    0
        ..
36256    1
36257   -1
35654    0
36258    0
36259    0
Length: 50032, dtype: int64
12814    1
11499    0
12815   -1
5008     0
11500    0
        ..
36256    1
36257   -1
35654    0
36258    0
36259    0
Length: 50032, dtype: int64
12814    1
11499    0
12815   -1
5008     0
11500    0
        ..
36256    1
36257   -1
35654    0
36258    0
36259    0
Length: 50032, dtype: int64
12814    1
11499    0
12815   -1
5008     0
11500    0
        ..
36256    1
36257   -1
35654    0
36258    0
36259    0
Length: 50032, dtype: int64
12814    1
11499    0
12815   -1
5008     0
11500    0
        ..
36256    1
36257   -1
35654    0
36258  

In [166]:
table2.dtypes

UserName                        object
OutletCode                      object
user_id                          int64
date_id                          int64
is_reported                      int64
date_format             datetime64[ns]
SessionUid                      object
Question                        object
Response                        object
month                            int64
day                              int64
Enfriador Coca-Cola              int64
Enfriador Detallista             int64
Coca-Cola                        int32
Detallista                       int32
OutletCodeChanged               object
Flag_CC                          int64
dtype: object

In [167]:
for OutletCodeChanged in table2:
      s = table2['OutletCodeChanged'].unique()
      for i in s:
          if i == 'True':
              Flag_Det = 'change'
              
          else: 
              Flag_Det = table2["Enfriador Detallista"] - table2["Detallista"]

              table2['Flag_Det']=Flag_Det
              
      print(Flag_Det)      

12814    0
11499    0
12815    1
5008     0
11500    0
        ..
36256   -1
36257    1
35654    0
36258    0
36259    0
Length: 50032, dtype: int64
12814    0
11499    0
12815    1
5008     0
11500    0
        ..
36256   -1
36257    1
35654    0
36258    0
36259    0
Length: 50032, dtype: int64
12814    0
11499    0
12815    1
5008     0
11500    0
        ..
36256   -1
36257    1
35654    0
36258    0
36259    0
Length: 50032, dtype: int64
12814    0
11499    0
12815    1
5008     0
11500    0
        ..
36256   -1
36257    1
35654    0
36258    0
36259    0
Length: 50032, dtype: int64
12814    0
11499    0
12815    1
5008     0
11500    0
        ..
36256   -1
36257    1
35654    0
36258    0
36259    0
Length: 50032, dtype: int64
12814    0
11499    0
12815    1
5008     0
11500    0
        ..
36256   -1
36257    1
35654    0
36258    0
36259    0
Length: 50032, dtype: int64
12814    0
11499    0
12815    1
5008     0
11500    0
        ..
36256   -1
36257    1
35654    0
36258  

In [168]:
table2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50032 entries, 12814 to 36259
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   UserName              50032 non-null  object        
 1   OutletCode            50032 non-null  object        
 2   user_id               50032 non-null  int64         
 3   date_id               50032 non-null  int64         
 4   is_reported           50032 non-null  int64         
 5   date_format           50032 non-null  datetime64[ns]
 6   SessionUid            50032 non-null  object        
 7   Question              50032 non-null  object        
 8   Response              50032 non-null  object        
 9   month                 50032 non-null  int64         
 10  day                   50032 non-null  int64         
 11  Enfriador Coca-Cola   50032 non-null  int64         
 12  Enfriador Detallista  50032 non-null  int64         
 13  Coca-Cola   

In [169]:
table_final = table2[['UserName','user_id','OutletCode','date_id', 'is_reported', 'SessionUid', 'Question', 'Response', 'month', 'day', 'OutletCodeChanged', 'Enfriador Coca-Cola', 'Coca-Cola', 'Flag_CC', 'Enfriador Detallista', 'Detallista', 'Flag_Det']]
table_final

,UserName,user_id,OutletCode,date_id,is_reported,SessionUid,Question,Response,month,day,OutletCodeChanged,Enfriador Coca-Cola,Coca-Cola,Flag_CC,Enfriador Detallista,Detallista,Flag_Det
12814,cdmx26@storecheck.com,47713,100000195,20210312,1,78f994cc-4a41-46ab-b0bb-8b11323aac57,Tipo Enfriador Detallista,Vitrina,3,12,False,1,0,1,0,0,0
11499,cdmx24@storecheck.com,45848,100000208,20210228,1,1ef2922f-4c61-47ab-a065-e94bc750a97b,Tipo Enfriador Detallista,Vitrina,2,28,True,1,1,0,0,0,0
12815,cdmx26@storecheck.com,47713,100000208,20210312,1,1061c10c-0134-4c44-ac84-ba4ae39b248e,Tipo Enfriador Detallista,Vitrina,3,12,False,0,1,-1,1,0,1
5008,cdmx13@storecheck.com,45836,100000283,20210326,1,dabf8dd8-3a0c-4456-b4ae-0cce7fcadb62,Tipo Enfriador Detallista,Vitrina,3,26,True,0,0,0,1,1,0
11500,cdmx24@storecheck.com,45848,100000390,20210611,1,d0b07c3d-e1d8-4674-828f-d5628c9a6c00,Tipo Enfriador Detallista,Vitrina,6,11,True,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36256,merida4@storecheck.com,45796,99361,20210122,1,d9689f04-47bc-4ca1-9748-6d168b1ab902,Tipo Enfriador Detallista,Enfriador Vertical,1,22,True,1,0,1,0,1,-1
36257,merida4@storecheck.com,45796,99361,20210412,1,fe842bae-8ee8-49fb-b2c1-1c05d5fe7a53,Tipo Enfriador Detallista,Enfriador Vertical,4,12,False,0,1,-1,1,0,1
35654,merida3@storecheck.com,45795,99750,20210104,1,9537a728-a264-4603-92d8-64e754403dc0,Tipo Enfriador Detallista,Cámara Fría,1,4,True,0,0,0,1,1,0
36258,merida4@storecheck.com,45796,99750,20210208,1,01ddcc9b-0294-4658-b0f5-4d004035e31c,Tipo Enfriador Detallista,Enfriador Vertical,2,8,False,0,0,0,1,1,0


In [170]:
query = """ 

select distinct sl.SessionUId, sl.LocationId from dbo.SessionList sl
where YEAR(sl.VisitStartDateTime) = '2021' 
 

        """

In [171]:
dff = pd.DataFrame(get_query(query))
#df = df.sort_values(by="VisitStartDateTime",ascending=False)
dff=dff.rename(columns={"SessionUId": "SessionUid"})
dff.dtypes

SessionUid    object
LocationId     int64
dtype: object

In [172]:
Ifi_historico = pd.merge(table_final, dff, on='SessionUid', how='inner')
Ifi_historico

,UserName,user_id,OutletCode,date_id,is_reported,SessionUid,Question,Response,month,day,OutletCodeChanged,Enfriador Coca-Cola,Coca-Cola,Flag_CC,Enfriador Detallista,Detallista,Flag_Det,LocationId
0,cdmx26@storecheck.com,47713,100000195,20210312,1,78f994cc-4a41-46ab-b0bb-8b11323aac57,Tipo Enfriador Detallista,Vitrina,3,12,False,1,0,1,0,0,0,6816944
1,cdmx24@storecheck.com,45848,100000208,20210228,1,1ef2922f-4c61-47ab-a065-e94bc750a97b,Tipo Enfriador Detallista,Vitrina,2,28,True,1,1,0,0,0,0,6828931
2,cdmx26@storecheck.com,47713,100000208,20210312,1,1061c10c-0134-4c44-ac84-ba4ae39b248e,Tipo Enfriador Detallista,Vitrina,3,12,False,0,1,-1,1,0,1,6828931
3,cdmx13@storecheck.com,45836,100000283,20210326,1,dabf8dd8-3a0c-4456-b4ae-0cce7fcadb62,Tipo Enfriador Detallista,Vitrina,3,26,True,0,0,0,1,1,0,6272009
4,cdmx24@storecheck.com,45848,100000390,20210611,1,d0b07c3d-e1d8-4674-828f-d5628c9a6c00,Tipo Enfriador Detallista,Vitrina,6,11,True,0,0,0,1,1,0,6826853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50027,merida4@storecheck.com,45796,99361,20210122,1,d9689f04-47bc-4ca1-9748-6d168b1ab902,Tipo Enfriador Detallista,Enfriador Vertical,1,22,True,1,0,1,0,1,-1,6294575
50028,merida4@storecheck.com,45796,99361,20210412,1,fe842bae-8ee8-49fb-b2c1-1c05d5fe7a53,Tipo Enfriador Detallista,Enfriador Vertical,4,12,False,0,1,-1,1,0,1,6294575
50029,merida3@storecheck.com,45795,99750,20210104,1,9537a728-a264-4603-92d8-64e754403dc0,Tipo Enfriador Detallista,Cámara Fría,1,4,True,0,0,0,1,1,0,6294577
50030,merida4@storecheck.com,45796,99750,20210208,1,01ddcc9b-0294-4658-b0f5-4d004035e31c,Tipo Enfriador Detallista,Enfriador Vertical,2,8,False,0,0,0,1,1,0,6294577


In [174]:
Ifi_historico=Ifi_historico[['UserName', 'user_id', 'OutletCode', 'LocationId', 'date_id', 'is_reported','SessionUid', 'Question', 'Response', 'month', 'day', 'OutletCodeChanged', 'Enfriador Coca-Cola', 'Flag_CC', 'Enfriador Detallista', 'Flag_Det']]

In [175]:
Ifi_historico.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50032 entries, 0 to 50031
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   UserName              50032 non-null  object
 1   user_id               50032 non-null  int64 
 2   OutletCode            50032 non-null  object
 3   LocationId            50032 non-null  int64 
 4   date_id               50032 non-null  int64 
 5   is_reported           50032 non-null  int64 
 6   SessionUid            50032 non-null  object
 7   Question              50032 non-null  object
 8   Response              50032 non-null  object
 9   month                 50032 non-null  int64 
 10  day                   50032 non-null  int64 
 11  OutletCodeChanged     50032 non-null  object
 12  Enfriador Coca-Cola   50032 non-null  int64 
 13  Flag_CC               50032 non-null  int64 
 14  Enfriador Detallista  50032 non-null  int64 
 15  Flag_Det              50032 non-null

In [176]:
import pandas as pd 
import pyodbc
import sqlalchemy
import datetime as dt 
import numpy as np

#Se definen las conexiones

def get_engine():
    connection_string = "mssql+pyodbc://{0}@{2}:{1}@{2}.database.windows.net:{4}/{3}?driver=ODBC+Driver+17+for+SQL+Server".format(
                        'jrivas',
                        'XXXXX',
                        'koice',
                        'ko_ice',
                        1433)
    res = sqlalchemy.engine.create_engine(connection_string, connect_args={'autocommit': True})
    return res


def get_query(query):
    engine = get_engine()
    res = pd.read_sql(query, con=engine)
    return res


In [177]:
engine = get_engine()
Ifi_historico.to_sql('Primera_Posicion_tt', con=engine, schema='reportes', if_exists='append', index=False)